In [1]:
#! pip install pandas

import re
import pandas as pd

In [2]:
statistics_template = {'id':[], 'value':[], 'nano_delta':[]}


def parse_log(filename):
    statistics_regexp = re.compile(r".*<<\{(\d+),\s+(\d+.\d*),\s(\d+)}>>.*")
    statistics = statistics_template.copy()
    
    with open(f'./resources/{filename}', 'r') as log_file:
        line = log_file.readline()
        while line:
            groups = re.search(statistics_regexp, line)
            if groups:
                _id, value, nano_delta = groups.groups()
                statistics['id'].append(_id)
                statistics['value'].append(value)
                statistics['nano_delta'].append(nano_delta)
            line = log_file.readline()
    
    df_statistics = pd.DataFrame.from_dict(statistics).astype({'id': int, 'value': float, 'nano_delta': int})
    df_statistics["micro_delta"] = df_statistics["nano_delta"] // 1e3
    return df_statistics

In [3]:
runs = {
    "first try": parse_log("first_try.txt")
}

In [4]:
runs["1m runs"] = parse_log("1million.txt")

In [5]:
overall_stats = {'name':[], 'alerts_received':[], 'fastest':[], '0.5q':[],'0.9q':[],'0.95q':[],'0.99q':[],'slowest':[]}
for _name, stats in runs.items():
    _df = stats[["micro_delta"]]
    overall_stats['name'].append(_name)
    overall_stats['alerts_received'].append(len(_df))
    overall_stats['fastest'].append(_df.min().iloc[0])
    overall_stats['0.5q'].append(_df.quantile(0.5).iloc[0])
    overall_stats['0.9q'].append(_df.quantile(0.9).iloc[0])
    overall_stats['0.95q'].append(_df.quantile(0.95).iloc[0])
    overall_stats['0.99q'].append(_df.quantile(0.99).iloc[0])
    overall_stats['slowest'].append(_df.max().iloc[0])
pd.DataFrame.from_dict(overall_stats)

,name,alerts_received,fastest,0.5q,0.9q,0.95q,0.99q,slowest
0,first try,8653,667.0,2996.0,3877.8,4109.0,4615.0,21341.0
1,1m runs,443810,224.0,1662.0,2564.0,2839.0,3535.0,21341.0
